# Construction du modèle de reconnaissance de caractère basé sur le dataset MIST

Dans ce notebook sera présenté la création du modèle qui permettra de **classifier** les lettres de l'alphabet de a-z et A-Z.

Pour ce faire nous allons utiliser le dataset d'images prétraités créés par le notebook [NIST-preprocessing](/notebooks/notebooks/character_recognition/NIST-preprocessing.ipynb)

Pour exécuter ce notebook, veillez à ce que le jeu de données soit bien sous la forme suivante.
- data
    - processed
         - NIST-dataset
            - train
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png
            - test_set
                - a000001.png
                - a000002.png
                ...
                ...
                - a00000n.png


## Importation des dépendances

In [ ]:
import os
from string import ascii_lowercase, ascii_uppercase

import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Sélection des devices

In [ ]:
from tensorflow.python.client import device_lib

print("Num of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_lib.list_local_devices()

In [ ]:
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)

## Initialisation de tensorboard

In [ ]:
%load_ext tensorboard

## Définition des paramètres du notebook

In [ ]:
NIST_PROCESSED_PATH = "../../data/processed/NIST-dataset"
TRAIN_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "train", "nist_processed_train.csv")
TEST_SET_PATH = os.path.join(NIST_PROCESSED_PATH, "test", "nist_processed_test.csv")
IMG_SIZE = 41

## Importation du dataset

Dans cette section, nous allons charger le dataset afin d'obtenir un train_data_set, train_label_set, ainsi qu'un test_data_set et un test_label_set

### Chargement du dataset depuis le disque

In [ ]:
train_set_frame = pd.read_csv(TRAIN_SET_PATH)
test_set_frame = pd.read_csv(TEST_SET_PATH)

In [ ]:
def get_y_original_shape(y_raw_set):
    x  = y_raw_set.shape
    return y_raw_set.reshape(x[0], 1)

def get_x_original_shape(x_raw_set):
    x, y = x_raw_set.shape
    return x_raw_set.reshape(x, IMG_SIZE, IMG_SIZE)

In [ ]:
def get_y_and_x_set_from_dataframe(dataframe):
    y_raw_set = dataframe["label"].to_numpy(dtype=str)
    x_raw_set = test_set_frame.iloc[:, 2:].to_numpy(dtype=np.uint8)

    y_set = get_y_original_shape(y_raw_set)
    x_set = get_x_original_shape(x_raw_set)

    return y_set, x_set

In [ ]:
y_train, x_train = get_y_and_x_set_from_dataframe(train_set_frame)
y_test, x_test = get_y_and_x_set_from_dataframe(test_set_frame)

print(y_train.shape)
print(x_train.shape)

## Encodage des labels

On récupère tous les labels possibles

In [ ]:
label_list = [char for char in ascii_lowercase + ascii_uppercase]
print(label_list[0:5])

On définit l'encoder à utiliser

In [ ]:
encoder = LabelBinarizer()
encoder.fit(label_list)

On encode nos données au format OneShot

In [ ]:
y_encoded_train = encoder.transform(y_train)
y_encoded_test = encoder.transform(y_test)
y_encoded_train.shape

## Création du modèle de classification

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(41,41)),
    layers.Flatten(input_shape=(41, 41)),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(100, activation="relu"),
    layers.Dense(52, activation="softmax"),
])

model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=["acc", "categorical_crossentropy"])
model.summary()

## Configuration de tensorboard si installé

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

## Entrainement du modèle

In [ ]:
model.fit(x=x_train, y=y_encoded_train, epochs=20, callbacks=[tensorboard_callback])

## Test du modèle sur le jeu de test

In [ ]:
model.evaluate(x_test, y_encoded_test, verbose=2)

In [ ]:
%tensorboard --logdir {log_dir}